In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
from functools import reduce

In [224]:
#importing data files for all the months separately
#POWER
path=r"./Raw Data/"
df_p=[pd.read_csv(file,index_col="ts") for file in glob.glob(path+ "/*.csv")]
#TEMP
path=r"/Users/chandu/Desktop/IOT DATATHON/Chiller_one/T"
df_t=[pd.read_csv(file,index_col="ts") for file in glob.glob(path+ "/*.csv")]
#EVAPORATOR
path=r"/Users/chandu/Desktop/IOT DATATHON/Chiller_one/E"
df_e=[pd.read_csv(file,index_col="ts") for file in glob.glob(path+ "/*.csv")]
#CONDENSER
path=r"/Users/chandu/Desktop/IOT DATATHON/Chiller_one/C"
df_c=[pd.read_csv(file,index_col="ts") for file in glob.glob(path+ "/*.csv")]


In [227]:
#concatenating the dataframes and subsetting the columns
tab_p=pd.concat(df_p).sort_index()
tab_p=tab_p[["ch1Watt","ch2Watt","ch3Watt","totalPositiveWattHour"]]

tab_t=pd.concat(df_t).sort_index()
tab_t=tab_t[["value1","value2","value3","value4"]]
tab_t.columns=["temp1","temp2","temp3","temp4"]

tab_e=pd.concat(df_e).sort_index()
tab_e=tab_e[["flowRate","flowSpeed","totalFlowRate"]]
tab_e.columns=["e.flowRate","e.flowSpeed","e.totalFlowRate"]

tab_c=pd.concat(df_c).sort_index()
tab_c=tab_c[["flowRate","flowSpeed","totalFlowRate"]]
tab_c.columns=["c.flowRate","c.flowSpeed","c.totalFlowRate"]


In [228]:
#creating a list for dataframes
tables=[tab_p,tab_t,tab_e,tab_c]

In [229]:
#changing the index to datetimeindex
for i in tables:
    i.index=pd.DatetimeIndex(i.index)
    i.index=i.index.map(lambda x:x.replace(second=0,microsecond=0))

In [230]:
print(tab_p.shape,tab_t.shape,tab_e.shape,tab_c.shape)

(295954, 4) (301943, 4) (293430, 3) (293416, 3)


In [231]:
#removing duplicates
tab_p=tab_p.reset_index().drop_duplicates(subset="ts",keep="first").set_index("ts")
tab_t=tab_t.reset_index().drop_duplicates(subset="ts",keep="first").set_index("ts")
tab_e=tab_e.reset_index().drop_duplicates(subset="ts",keep="first").set_index("ts")
tab_c=tab_c.reset_index().drop_duplicates(subset="ts",keep="first").set_index("ts")

In [232]:
#creating a new index
all_days = pd.date_range(tab_t.index.min(), tab_t.index.max(), freq='T')

In [233]:
#reindexing with missing time indices
tab_p=tab_p.reindex(all_days)
tab_t=tab_t.reindex(all_days)
tab_e=tab_e.reindex(all_days)
tab_c=tab_c.reindex(all_days)

In [234]:
#merging the dataframes
c1_final=tab_p.merge(tab_t,left_index=True, right_index=True).merge(tab_e,left_index=True, right_index=True).merge(tab_c,left_index=True, right_index=True)

In [238]:
c1_final.head()

,ch1Watt,ch2Watt,ch3Watt,totalPositiveWattHour,temp1,temp2,temp3,temp4,e.flowRate,e.flowSpeed,e.totalFlowRate,c.flowRate,c.flowSpeed,c.totalFlowRate
2017-05-01 00:00:00,349.0,0.0,353.0,27545200.0,18.068238,20.535583,23.376769,23.387453,0.000000,0.00000,124807.0,0.000000,0.000000,190527.0
2017-05-01 00:01:00,383.0,0.0,396.0,27545200.0,16.166991,16.914672,22.565002,22.639772,0.000000,0.00000,124807.0,35.284023,0.303604,190527.0
2017-05-01 00:02:00,379.0,0.0,394.0,27545200.0,15.964052,16.722413,22.372743,22.351379,0.000000,0.00000,124807.0,144.330154,1.241898,190529.0
2017-05-01 00:03:00,389.0,0.0,408.0,27545200.0,17.053528,17.822570,22.265932,22.372743,10.812524,0.16108,124807.0,195.770782,1.684522,190532.0
2017-05-01 00:04:00,5748.0,4480.0,4942.0,27545300.0,18.559570,19.531554,22.415465,22.490235,77.101921,1.14863,124808.0,214.631348,1.846809,190535.0
